In [5]:
# -*- coding: utf-8 -*-
from ekonlpy.sentiment import MPCK

import pandas as pd
import multiprocessing as mp
import pickle

import os, sys
import re

import preprocessing_mp_worker as worker

# Tidy BOK minutes text

Jupyter Lab에서의 multiprocessing 사용을 위해 main함수인 preprocessing()를 제외한 모든 함수는 preprocessing_mp_worker.py에 옮겨졌다. 

Code Attribution: 김현진T, Fininsight

### Korean stop words from here: 
https://www.ranks.nl/stopwords/korean

## Test code: 어떤 식으로 파싱되는지 확인. 

In [2]:
source_folder="./data_files/BOK_minutes/pdf2txt/"
output_file="./data_files/BOK_minutes/pkl/"

txt_files = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]
txt_files = [f for f in txt_files if f[-4:] == '.txt']
txt_files.sort()

In [3]:
df = pd.DataFrame(columns=['date', 'minutes'])
df["Economic Situation"] = ""
df["Foreign Currency"] = ""
df["Financial Markets"] = ""
df["Monetary Policy"] = ""
df["Participant Views"] = ""
df["Government View"] = ""

df["Economic Situation count"] = ""
df["Foreign Currency count"] = ""
df["Financial Markets count"] = ""
df["Monetary Policy count"] = ""
df["Participant Views count"] = ""
df["Government View count"] = ""
df['ngrams'] =""

### 한 문서의 ngram의 갯수가 매우 많다. 이를 모두 각각 하나의 row로 만들면 df의 길이가 매우 커진다. 

* 0번째 문서의 모든 ngram을 ,(comma)로 separate한 경우 (원본코드의 방법)

In [44]:
%%time

for txt_file in txt_files[:1]:
    with open(source_folder + txt_file, 'r',encoding='utf-8') as f :
        txt = f.read()
        sections, section_texts = worker.preprocess_minutes(txt)

        print(txt_file)

        ngrams = []
        for i, section_text in enumerate(section_texts[1:3]):
            print("doing #{} iteration...".format(i))
            with mp.Pool(processes=6) as p:
                mp_result = p.map(worker.text2ngram, section_text)
            for i in mp_result:
                ngrams += i

    df.loc[len(df)] = [txt_file.split('_-_')[0],
                       txt,
                       '@@@'.join(section_texts[0]),
                       '@@@'.join(section_texts[1]),
                       '@@@'.join(section_texts[2]),
                       '@@@'.join(section_texts[3]),
                       '@@@'.join(section_texts[4]),
                       '@@@'.join(section_texts[5]),
                       len(section_texts[0]),
                       len(section_texts[1]),
                       len(section_texts[2]),
                       len(section_texts[3]),
                       len(section_texts[4]),
                       len(section_texts[5]),
                       '@@@'.join(ngrams)]

2005-06-09_-_금융통화위원회 의사록(2005년도 제12차).txt
doing #0 iteration...
doing #1 iteration...
Wall time: 17.8 s


In [45]:
df.memory_usage(index=True, deep=True)

Index                           20992
date                           175808
minutes                     111929344
Economic Situation           21747712
Foreign Currency              5851520
Financial Markets            80897920
Monetary Policy                154816
Participant Views              154816
Government View                154816
Economic Situation count        94464
Foreign Currency count          94464
Financial Markets count         94464
Monetary Policy count           83968
Participant Views count         83968
Government View count           83968
ngrams                         448233
dtype: int64

In [46]:
df[['date', 'ngrams']].memory_usage(index=True, deep=True).sum()

645033

In [47]:
df.ngrams

0       경제/NNG,경제/NNG,비해/VV,특별/NNG,요인/NNG,불구/NNG,원엔/NN...
1                                                        
2                                                        
3                                                        
4                                   경@@@제@@@/@@@N@@@N@@@G
                              ...                        
2619                        프@@@리@@@미@@@엄@@@/@@@N@@@N@@@G
2620                                수@@@준@@@/@@@N@@@N@@@G
2621                                유@@@지@@@/@@@N@@@N@@@G
2622                                양@@@호@@@/@@@N@@@N@@@G
2623    경제/NNG@@@경제/NNG@@@비해/VV@@@특별/NNG@@@요인/NNG@@@불구...
Name: ngrams, Length: 2624, dtype: object

In [48]:
print(len(df.ngrams[0]))
df.ngrams[0][:300]

21924


'경제/NNG,경제/NNG,비해/VV,특별/NNG,요인/NNG,불구/NNG,원엔/NNG,환율/NNG,지속/NNG,하락/NNG,이유/NNG,대해/VV,위안/NNG,상시/NNG,통화/NNG,절상/NNG,기대/NNG,원화/NNG,달러/NNG,엔달러/NNG,환율/NNG,상승/NNG,추수/NNG,못하/VX,외환/NNG,수급/NNG,공급우위/NNG,상태/NNG,지속/NNG,국제투자은행/NNG,엔달러/NNG,환율/NNG,전망/NNG,상향/NNG,불구/NNG,원달러/NNG,조정/NNG,아직/MAG,이루/VV,기업/NNG,기대/NNG,원화/NNG,절'

* 0번째 문서의 모든 ngram을 각각 row로 만들어 df에 담았을 경우

In [49]:
%%time

for txt_file in txt_files[:1]:
    with open(source_folder + txt_file, 'r',encoding='utf-8') as f :
        txt = f.read()
        sections, section_texts = worker.preprocess_minutes(txt)

        print(txt_file)

        ngrams = []
        for i, section_text in enumerate(section_texts[1:3]):
            print("doing #{} iteration...".format(i))
            with mp.Pool(processes=6) as p:
                mp_result = p.map(worker.text2ngram, section_text)
                for ngram_list in mp_result:
                    for item in ngram_list:
                        df.loc[len(df)] = [txt_file.split('_-_')[0],
                                   txt,
                                   '@@@'.join(section_texts[0]),
                                   '@@@'.join(section_texts[1]),
                                   '@@@'.join(section_texts[2]),
                                   '@@@'.join(section_texts[3]),
                                   '@@@'.join(section_texts[4]),
                                   '@@@'.join(section_texts[5]),
                                   len(section_texts[0]),
                                   len(section_texts[1]),
                                   len(section_texts[2]),
                                   len(section_texts[3]),
                                   len(section_texts[4]),
                                   len(section_texts[5]),
                                   '@@@'.join(item)]


2005-06-09_-_금융통화위원회 의사록(2005년도 제12차).txt
doing #0 iteration...
doing #1 iteration...
Wall time: 57.2 s


In [50]:
df.memory_usage(index=True, deep=True)

Index                           41944
date                           351281
minutes                     223645408
Economic Situation           43453984
Foreign Currency             11691890
Financial Markets           161641690
Monetary Policy                309337
Participant Views              309337
Government View                309337
Economic Situation count       188748
Foreign Currency count         188748
Financial Markets count        188748
Monetary Policy count          167776
Participant Views count        167776
Government View count          167776
ngrams                         798613
dtype: int64

In [51]:
df[['date', 'ngrams']].memory_usage(index=True, deep=True).sum()

1191838

In [52]:
df.ngrams

0       경제/NNG,경제/NNG,비해/VV,특별/NNG,요인/NNG,불구/NNG,원엔/NN...
1                                                        
2                                                        
3                                                        
4                                   경@@@제@@@/@@@N@@@N@@@G
                              ...                        
5238                            카@@@드@@@채@@@/@@@N@@@N@@@G
5239                        프@@@리@@@미@@@엄@@@/@@@N@@@N@@@G
5240                                수@@@준@@@/@@@N@@@N@@@G
5241                                유@@@지@@@/@@@N@@@N@@@G
5242                                양@@@호@@@/@@@N@@@N@@@G
Name: ngrams, Length: 5243, dtype: object

In [9]:
def preprocessing(source_folder="./data_files/BOK_minutes/pdf2txt/", output_file="./data_files/BOK_minutes/pkl/") :
    # 지정 폴더 내 파일 목록 조회 (파일만)
    txt_files = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]
    txt_files = [f for f in txt_files if f[-4:] == '.txt']
    txt_files.sort()
    df = pd.DataFrame(columns=['date', 'minutes'])
    df["Economic Situation"] = ""
    df["Foreign Currency"] = ""
    df["Financial Markets"] = ""
    df["Monetary Policy"] = ""
    df["Participant Views"] = ""
    df["Government View"] = ""

    df["Economic Situation count"] = ""
    df["Foreign Currency count"] = ""
    df["Financial Markets count"] = ""
    df["Monetary Policy count"] = ""
    df["Participant Views count"] = ""
    df["Government View count"] = ""
    df['ngrams'] =""
    

    for txt_file in txt_files :
        try :
            with open(source_folder + txt_file, 'r',encoding='utf-8') as f :
                txt = f.read()
                sections, section_texts = preprocess_minutes(txt)

                print(txt_file)

                ngrams = []
                for section_text in section_texts[1:3] :
                    for sentence in section_text :
                        ngrams.append(','.join(text2ngram(sentence)))

                #print('@@@'.join(ngrams))
                #break

                df.loc[len(df)] = [txt_file.split('_-_')[0],
                                   txt,
                                   '@@@'.join(section_texts[0]),
                                   '@@@'.join(section_texts[1]),
                                   '@@@'.join(section_texts[2]),
                                   '@@@'.join(section_texts[3]),
                                   '@@@'.join(section_texts[4]),
                                   '@@@'.join(section_texts[5]),
                                   len(section_texts[0]),
                                   len(section_texts[1]),
                                   len(section_texts[2]),
                                   len(section_texts[3]),
                                   len(section_texts[4]),
                                   len(section_texts[5]),
                                   '@@@'.join(ngrams)]
        except :
            print('Error occurred at {}'.format(txt_file))
            print('Error: ', sys.exc_info()[0])
            pass
    
    df.to_pickle(output_file)
        
    print(df.shape)
    print("---")
    print(df.head())

In [ ]:
preprocessing()

2005-06-09_-_금융통화위원회 의사록(2005년도 제12차).txt
